In [2]:
import os
import pandas as pd

In [3]:
PRJ_DIR = os.path.join('/home', 'jovyan', 'work')
MAIN_SRC = os.path.join(PRJ_DIR, 'main.cpp')

In [4]:
expand_file_list = []

with open(MAIN_SRC, 'r') as f:
    lines = f.readlines()
    
    for line in lines:
        line = line.strip()
        
        if "#include" in line and '"' in line:
            file = line.replace("#include", "")
            file = file.replace('"', "")
            file = file.replace(' ', "")
            
            expand_file_list.append(file)
            
            file = file.replace(".hpp", ".cpp")
            expand_file_list.append(file)
            
expand_file_list.append('main.cpp')

In [5]:
expand_file_list

['Utility.hpp',
 'Utility.cpp',
 'XorShift.hpp',
 'XorShift.cpp',
 'Parameter.hpp',
 'Parameter.cpp',
 'SearchInfo.hpp',
 'SearchInfo.cpp',
 'State.hpp',
 'State.cpp',
 'main.cpp']

In [6]:
file_list = []

for f in expand_file_list:
    if '.hpp' in f:
        file_list.append(f)

for f in expand_file_list:
    if '.cpp' in f:
        file_list.append(f)

In [7]:
expand_file_list = file_list

In [8]:
expanded_source = ["#define NDEBUG\n"]

# debugコマンドをオフ
debug_list = ['debug', 'debug2', 'debug_st', 'debug_sa', 'debug_bm', 'debug_gd', 'debug_hc']

for debug_cmd in debug_list:
    expanded_source.append("#define {}(...) ;\n".format(debug_cmd))

chmax_def = False
chmin_def = False

for file in expand_file_list:
    debug_file = False
    
    for debug_cmd in debug_list:
        if '{}.cpp'.format(debug_cmd) in file:
            debug_file = True
            print('Debug file: ' + file)
    
    if debug_file:
        continue
    
    file_name = os.path.join(PRJ_DIR, file)
    
    with open(file_name, 'r') as f:
        for line in f.readlines():
            if "#include" in line and '"' in line:
                continue
                
            if "#pragma once" in line:
                continue
                
            if 'bool chmax' in line:
                if chmax_def:
                    continue
                else:
                    chmax_def = True
                    
            if 'bool chmin' in line:
                if chmin_def:
                    continue
                else:
                    chmin_def = True
                    
            expanded_source.append(line)

    expanded_source.append("\n")

## combined.cppの生成

In [44]:
with open("../combined.cpp", "w") as f:
    for line in expanded_source:
        f.writelines(line)

## Makefileの生成

In [9]:
source_file_list = []

for source_file in expand_file_list:
    if '.hpp' in source_file:
        continue
        
    source_file_list.append(source_file.replace('.cpp', ''))

In [10]:
source_file_list

['Utility', 'XorShift', 'Parameter', 'SearchInfo', 'State', 'main']

In [11]:
make_file_lines = []

make_file_lines.append('CC = ccache g++')
make_file_lines.append('#CFLAGS = -g -Wall -std=c++17 -O0 -fsanitize=undefined,address -DLOCAL')
make_file_lines.append('')
make_file_lines.append('# core dump出力用')
make_file_lines.append('#CFLAGS = -g -Wall --std=c++17 -O0 -DLOCAL')
make_file_lines.append('')
make_file_lines.append('CFLAGS = -Wall --std=c++17 -O2 -DLOCAL')
make_file_lines.append('#CFLAGS = -pg -g -Wall --std=c++17 -O0 -DLOCAL')
make_file_lines.append('')

# target objects
make_str = "ALL:"

for source_file in source_file_list:
    make_str += " {}.o".format(source_file)
    
make_file_lines.append(make_str)
make_file_lines.append("\t$(CC) $(CFLAGS) -o main \\")

for source_file in source_file_list:
    make_file_lines.append("\t{}.o \\".format(source_file))

make_file_lines.append('')
make_file_lines.append('clean:')
make_file_lines.append('\trm main *.o')

make_file_lines.append('')
make_file_lines.append('run:')
make_file_lines.append('\t./main')

make_file_lines.append('')
make_file_lines.append('.cpp.o:')
make_file_lines.append('\t$(CC) $(CFLAGS) -o $@ -c $<')
make_file_lines.append('')


In [34]:
with open("../Makefile.gen", "w") as f:
    for line in make_file_lines:
        f.writelines(line + '\n')

## サブディレクトリ用Makefile(Optunaチューニングetc)

In [15]:
make_file_lines = []

make_file_lines.append('CC = ccache g++')
make_file_lines.append('#CFLAGS = -g -Wall -std=c++17 -O0 -fsanitize=undefined,address -DLOCAL')
make_file_lines.append('')
make_file_lines.append('# core dump出力用')
make_file_lines.append('#CFLAGS = -g -Wall --std=c++17 -O0 -DLOCAL')
make_file_lines.append('')
make_file_lines.append('CFLAGS = -Wall --std=c++17 -O2 -DLOCAL')
make_file_lines.append('#CFLAGS = -pg -g -Wall --std=c++17 -O0 -DLOCAL')
make_file_lines.append('')

# target objects
make_str = "ALL:"

for source_file in source_file_list:
    make_str += " {}.o".format(source_file)
    
make_file_lines.append(make_str)
make_file_lines.append("\t$(CC) $(CFLAGS) -o main \\")

for source_file in source_file_list:
    make_file_lines.append("\t{}.o \\".format(source_file))

make_file_lines.append('')
make_file_lines.append('clean:')
make_file_lines.append('\trm main *.o')

make_file_lines.append('')
make_file_lines.append('run:')
make_file_lines.append('\t./main')

make_file_lines.append('')

for source_file in source_file_list:
    if "main" in source_file:
        make_file_lines.append("{}.o: {}.cpp".format(source_file, source_file))
    else:
        make_file_lines.append("{}.o: ../{}.cpp".format(source_file, source_file))
    
    make_file_lines.append('\t$(CC) $(CFLAGS) -o $@ -c $<')
    make_file_lines.append('')

make_file_lines.append('.cpp.o:')
make_file_lines.append('\t$(CC) $(CFLAGS) -o $@ -c $<')
make_file_lines.append('')


In [16]:
with open("../Makefile_sub.gen", "w") as f:
    for line in make_file_lines:
        f.writelines(line + '\n')